Due to challenges faced trying to access the blockchain data en masse, i created a local instance of the lbry-explorer and this notebook leverages the mongoDB data source that exists as part of the lbry-explorer.

In [1]:
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [2]:
import getpass
password = getpass.getpass()

········


In [3]:
def connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = connect_mongo(host=host, port=port, username=username, password=password, db=db)
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find()
    #print list(cursor)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df.drop("__v", axis=1)

Collections:
    addresses,
    markets,
    richlists,
    stats,
    txes

In [4]:
db = "explorerdb"

In [5]:
address_df = read_mongo(db, "addresses", host='localhost', port=27017, username="zargham", password=password)

In [6]:
address_df[address_df.a_id.map(lambda s: bool((s[0] != "b") & (s[0] !="r")))].head()

,a_id,balance,received,sent,txs
0,coinbase,0,0,9.244448e+14,"[{u'type': u'vin', u'addresses': u'5dda81f4219..."


In [8]:
def get_label(address):
    if address in labels.keys():
        return labels[address]["label"]
    else:
        return "unknown"

In [9]:
address_df["label"] = address_df.a_id.map(get_label)

In [10]:
address_df[address_df.label != "unknown"]

,a_id,balance,received,sent,txs,label
404,rEQKyb7nd7UUGyEEn5xRkk1fgXdTCf2ZCg,0.000000e+00,1.999000e+16,1.999000e+16,"[{u'type': u'vout', u'addresses': u'32f0ab6b2a...",LBRY Team
406,rMT5Sg8SyFP3ax2PRaweRCRZoMeYw4znEi,0.000000e+00,3.980000e+16,3.980000e+16,"[{u'type': u'vout', u'addresses': u'781c100a80...",Adoption Programs
410,r9DarmxyPjWkF7ocyxMzaNZN3a9gJvNTZJ,0.000000e+00,1.000000e+16,1.000000e+16,"[{u'type': u'vout', u'addresses': u'9600945091...",Strategic Partners
424,bHW58d37s1hBjj3wPBkn5zpCX3F8ZW3uWf,4.000000e+16,0.000000e+00,4.000000e+16,"[{u'type': u'vin', u'addresses': u'9a72043e11b...",LBRY Team Escrow (premine)
4057,bLPbiXBp6Vr3NSnsHzDsLNzoy5o36re9Cz,1.000090e+13,1.000090e+13,0.000000e+00,"[{u'type': u'vout', u'addresses': u'e2f9cb21d4...",Investor
4163,bFgohv4wprdCEtW2m5EVQyVNtxp4NxQMXi,0.000000e+00,7.352650e+13,7.352650e+13,"[{u'type': u'vout', u'addresses': u'7c77646740...",LBRY Hot Wallet
4833,bVUrbCK8hcZ5XWti7b9eNxKEBxzc1rr393,0.000000e+00,4.897770e+13,4.897770e+13,"[{u'type': u'vout', u'addresses': u'ed8fc03abd...",LBRY Hot Wallet
128905,bDzuP6Sawe7tadVKR7dmPW1JFYK9ELnjra,1.450000e+14,2.984199e+14,1.534199e+14,"[{u'type': u'vout', u'addresses': u'554d8fed52...",LBRY Hot Wallet
129116,bWbFyN8CJAghwavfRDuEfmv71swkapBcuy,0.000000e+00,8.419899e+12,8.419899e+12,"[{u'type': u'vout', u'addresses': u'b72ffc5bb9...",LBRY Hot Wallet
236796,rVRMmGLZenVXpT1NwYMGcnwFc642kEgWTV,1.000000e+16,1.000000e+16,1.000000e+06,"[{u'type': u'vout', u'addresses': u'73df395c9f...",Strategic Partners


In [11]:
tx_df = read_mongo(db, "txes", host='localhost', port=27017, username="zargham", password=password)

In [12]:
import pytz
tz = pytz.timezone('America/Montreal')
tx_df["datetime"] = tx_df.timestamp.map(lambda t: datetime.datetime.fromtimestamp(int(t), tz))

In [13]:
#jacks reward function
COIN = 10**8
block_time = 150
blocks_per_ramp = 100
start_ramp_height = 5001
limit = 1083221000 * COIN
subsidy_interval = 1 << 5
max_reductions = 500
peak_reward = 500 * COIN
premine = 400000000 * COIN


def reward(n):
    assert n >= 0
    if n < 5001:
        return COIN
    elif n < 55001:
        r = 1 + ((n - ((n - start_ramp_height) % blocks_per_ramp) - start_ramp_height) / blocks_per_ramp)
        return r * COIN
    else:
        level = int((n - 55001) / subsidy_interval)
        reduction = int((-1 + int((8 * level) + 1) ** 0.5) / 2)
        while level >= ((reduction * reduction + reduction) >> 1):
            if ((reduction * reduction + reduction) >> 1) > level:
                reduction -= 1
            else:
                reduction += 1
        if reduction * COIN >= peak_reward:
            return 0
        return peak_reward - (reduction * COIN)

In [14]:
tx_df["reward"] = tx_df.blockindex.map(reward)/COIN #coin converts Deweys to LBC

In [62]:
cb_df=tx_df[tx_df.vin.map(lambda X: "coinbase" in [x['addresses'] for x in X] )]

In [64]:
cb_df.tail()

,blockhash,blockindex,timestamp,total,txid,vin,vout,datetime,reward,out_address,in_address,out_amount,in_amount,lbry_out,lbry_in
515575,923f40e68e4cce3f73d8cee24a9802600fe7a79bc0944c...,47933,1474329439,43000004520,ed1d55b396ed22ab9e5a17aee00c266230e3592c63a983...,"[{u'amount': 43000004520.0, u'addresses': u'co...","[{u'amount': 43000004520.0, u'addresses': u'bE...",2016-09-19 19:57:19-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
515577,0bcbc9f8c17f19382c864c10cc31b1493e1a5f15db2132...,47934,1474329462,43000000000,891c7a847f08adbc5110bacae3be872404d75cf863e84f...,"[{u'amount': 43000000000.0, u'addresses': u'co...","[{u'amount': 43000000000.0, u'addresses': u'bX...",2016-09-19 19:57:42-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
515578,d95e7b4ea395b74edb76cab43a433b5f6bc06065f46623...,47935,1474329602,43000234500,6d96d3084a6c3de9ebe4d92a7a732dab4c86ed6213b89b...,"[{u'amount': 43000234500.0, u'addresses': u'co...","[{u'amount': 43000234500.0, u'addresses': u'bE...",2016-09-19 20:00:02-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
515629,aa9cbff393139d36d7ca2afe7eb89a6d4d78f4505ec50a...,47936,1474330135,43000004520,6b6d48a1767b52e975409aea249d4ef1bd4725ed3d383d...,"[{u'amount': 43000004520.0, u'addresses': u'co...","[{u'amount': 43000004520.0, u'addresses': u'bE...",2016-09-19 20:08:55-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
515631,8850fafc3f56faed531ae6ec16e1535336f372f7ef3907...,47937,1474330243,43000061700,6507659cb237c5b26743167b0a7024e9687cb623bd36c4...,"[{u'amount': 43000061700.0, u'addresses': u'co...","[{u'amount': 43000061700.0, u'addresses': u'bY...",2016-09-19 20:10:43-04:00,430,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False


In [56]:
print tx_df.vout.map(lambda X: [x['addresses'] for x in X])+tx_df.vin.map(lambda X: [x['addresses'] for x in X] )

0            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
1            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
2            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
3            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
4            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
5            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
6            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
7            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
8            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
9            [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
10           [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
11           [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
12           [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
13           [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
14           [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
15           [bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m, coinbase]
16           [bZi1WEjGtsdAwuZTnNNTCAZLxh

In [65]:
tx_df.head()

,blockhash,blockindex,timestamp,total,txid,vin,vout,datetime,reward,out_address,in_address,out_amount,in_amount,lbry_out,lbry_in
0,decb9e2cca03a419fd9cca0cb2b1d5ad11b088f22f8f38...,1,1466646588,100000000,5dda81f42196d2647a173b008d729cf778d3f609adf846...,"[{u'amount': 100000000, u'addresses': u'coinba...","[{u'amount': 100000000, u'addresses': u'bZi1WE...",2016-06-22 21:49:48-04:00,1,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
1,28547a040490cc48c6c130622631bb52bc5388f98fc287...,2,1466646589,100000000,479dbab52d36a3223f0c4c2f2fc7fbdd44154917af3c65...,"[{u'amount': 100000000, u'addresses': u'coinba...","[{u'amount': 100000000, u'addresses': u'bZi1WE...",2016-06-22 21:49:49-04:00,1,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
2,c919ad56b3fdfc23f55bb9fba14d1211f3972ff63b2021...,3,1466646589,100000000,1a76ca5e68384568197beb1e6da885e10e1617a7364592...,"[{u'amount': 100000000, u'addresses': u'coinba...","[{u'amount': 100000000, u'addresses': u'bZi1WE...",2016-06-22 21:49:49-04:00,1,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
3,d557e01dffd10b7dddc84f7c3596176bd97cba36b3f67f...,4,1466646590,100000000,0cc8fe1387560146d4a315a6d89c117207940c5ba9b750...,"[{u'amount': 100000000, u'addresses': u'coinba...","[{u'amount': 100000000, u'addresses': u'bZi1WE...",2016-06-22 21:49:50-04:00,1,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False
4,4ca73d305d9dcde059c482593df71c75ca89a74967ed13...,5,1466646590,100000000,79c5eccaf593a860975f756660df64520e8666753b1ae8...,"[{u'amount': 100000000, u'addresses': u'coinba...","[{u'amount': 100000000, u'addresses': u'bZi1WE...",2016-06-22 21:49:50-04:00,1,bZi1WEjGtsdAwuZTnNNTCAZLxhHkiHec4m,coinbase,100000000,100000000,False,False


In [38]:
tx_df['out_address'] = 
tx_df['in_address'] = 

TypeError: list indices must be integers, not str

In [15]:
tx_df['out_amount']=
tx_df['in_amount'] = 

In [25]:
lbryKeys = labels.keys()
tx_df['lbry_out'] = tx_df.out_address.map(lambda x: bool(x in lbryKeys))
tx_df['lbry_in'] = tx_df.in_address.map(lambda x: bool(x in lbryKeys)) 

In [26]:
relevent_tx = tx_df[(tx_df.lbry_out==True) | (tx_df.lbry_in==True)]


In [28]:
lbryKeys

['rEQKyb7nd7UUGyEEn5xRkk1fgXdTCf2ZCg',
 'rTZF9RvfkLJ6hwCwERwFd7dppRMeddKjpe',
 'bLPbiXBp6Vr3NSnsHzDsLNzoy5o36re9Cz',
 'r9DarmxyPjWkF7ocyxMzaNZN3a9gJvNTZJ',
 'bWbFyN8CJAghwavfRDuEfmv71swkapBcuy',
 'rRmURzvpHTysU4xUSp9CXeosBTbYfKs7n5',
 'rVRMmGLZenVXpT1NwYMGcnwFc642kEgWTV',
 'bVUrbCK8hcZ5XWti7b9eNxKEBxzc1rr393',
 'rMT5Sg8SyFP3ax2PRaweRCRZoMeYw4znEi',
 'bDzuP6Sawe7tadVKR7dmPW1JFYK9ELnjra',
 'bFgohv4wprdCEtW2m5EVQyVNtxp4NxQMXi',
 'bHW58d37s1hBjj3wPBkn5zpCX3F8ZW3uWf']

In [27]:
relevent_tx

,blockhash,blockindex,timestamp,total,txid,vin,vout,datetime,reward,out_address,in_address,out_amount,in_amount,lbry_out,lbry_in


In [ ]:
tx_df.plot(x="datetime", y= "reward")
plt.ylabel("LBC")

In [ ]:
relevent_tx = pd.DataFrame([row[0] for row in address_df[address_df.label != "unknown"].txs])

In [ ]:
relevent_tx.addresses.values
#these are transaction IDs

We are going to use transaction ids, the blockchain database and the addresses table to create a history of the LBRY addresses named in "labels" and used for operating the business.

In [ ]:
tx_df.blockindex.max()